In [103]:
import matplotlib.pyplot as plt
import numpy as np 
import tensorflow as tf
import os 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

# plot_importance( )를 이용하여 feature 중요도 시각화
from xgboost import XGBRegressor, plot_importance
from lightgbm import LGBMRegressor, plot_importance

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Embedding, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras   import utils, Model
from tensorflow.python.client import device_lib
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing import sequence

import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [14]:
Popcorn_Data = pd.read_csv("dataset\PopCorn\labeledTrainData.tsv",  delimiter="\t")

In [15]:
Popcorn_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [19]:
Popcorn_Data.drop(columns = 'id', inplace= True) # 필요 없는 데이터 삭제 진행 
Popcorn_Data.head()

,sentiment,review
0,1,With all this stuff going down at the moment w...
1,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,0,The film starts with a manager (Nicholas Bell)...
3,0,It must be assumed that those who praised this...
4,1,Superbly trashy and wondrously unpretentious 8...


In [73]:
Review      = Popcorn_Data[['review']]      # 만약 데이터 프레임 형식으로 열을 하나 추출 하고 싶다면 다음과 같이 [['원하는열']] 처럼 사용한다. 
Sentiment   = Popcorn_Data[['sentiment']]
Review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  25000 non-null  object
dtypes: object(1)
memory usage: 195.4+ KB


In [90]:
#Review 데이터 깔끔하게 처리 
import re
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words("english")) 
lemmatizer = WordNetLemmatizer()


def clean_text(text):
    text = re.sub(r'[^\w\s]','',str(text), re.UNICODE)
    text = text.lower()
    text = [lemmatizer.lemmatize(token) for token in text.split(" ")]
    text = [lemmatizer.lemmatize(token, "v") for token in text]
    text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    # print(text)
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kch\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\kch\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [91]:
Review['Process_Review'] = Review.review.apply(lambda x : clean_text(x) )

In [92]:
Review.head()

,review,Process_Review
0,With all this stuff going down at the moment w...,stuff go moment mj ive start listen music watc...
1,"\The Classic War of the Worlds\"" by Timothy Hi...",classic war world timothy hines entertain film...
2,The film starts with a manager (Nicholas Bell)...,film start manager nicholas bell give welcome ...
3,It must be assumed that those who praised this...,must assume praise film greatest film opera ev...
4,Superbly trashy and wondrously unpretentious 8...,superbly trashy wondrously unpretentious 80 ex...


In [105]:
def text_data_Hotcoding(docs) : 
    token = Tokenizer()             # Token 화 시키기 
    token.fit_on_texts(docs)        # 각각의 문장을 token화 시킨다. 
    token.word_index                # 문장을 각각 index화 시킨다. 
    print(len(token.word_index))
    x = token.texts_to_sequences(docs) # 각각의 문장을 숫자로 변화 시킨다. 
    X_padded = sequence.pad_sequences(x, maxlen=130)  # 문장의 전체를 각각 일정하게 만들기 위해 padding 시켜 준다. 
    return X_padded

In [106]:
train = text_data_Hotcoding(Review['Process_Review'])

103051


In [116]:
def createLSTMModel():
    model = Sequential()
    model.add(Embedding(103052, 130))
    model.add(LSTM(130, activation='tanh'))
    model.add(Dense(1, activation='sigmoid'))
    return model

model = createLSTMModel()
model.reset_metrics()

In [117]:
X_train, X_test, Y_train, Y_test = train_test_split(train, Sentiment)

In [119]:
def trainModel(model, x_train, y_train, x_test, y_test):
    model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

    # 모델의 실행
    history = model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

    return model, history

trained_Model, history = trainModel(model, X_train, Y_train, X_test, Y_test )

Epoch 1/20
147/147 [==============================] - 24s 164ms/step - loss: 0.4212 - accuracy: 0.8072 - val_loss: 0.2890 - val_accuracy: 0.8851
Epoch 2/20
147/147 [==============================] - 24s 160ms/step - loss: 0.1614 - accuracy: 0.9431 - val_loss: 0.3224 - val_accuracy: 0.8733
Epoch 3/20
147/147 [==============================] - 23s 160ms/step - loss: 0.0638 - accuracy: 0.9798 - val_loss: 0.4184 - val_accuracy: 0.8664
Epoch 4/20
147/147 [==============================] - 24s 163ms/step - loss: 0.0334 - accuracy: 0.9903 - val_loss: 0.4855 - val_accuracy: 0.8538
Epoch 5/20
147/147 [==============================] - 25s 171ms/step - loss: 0.0197 - accuracy: 0.9942 - val_loss: 0.5212 - val_accuracy: 0.8680
Epoch 6/20
147/147 [==============================] - 25s 170ms/step - loss: 0.0269 - accuracy: 0.9923 - val_loss: 0.4783 - val_accuracy: 0.8576
Epoch 7/20
147/147 [==============================] - 23s 159ms/step - loss: 0.0251 - accuracy: 0.9916 - val_loss: 0.6178 - val_ac

KeyboardInterrupt: 

In [112]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 130)         13396630  
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               92400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 13,489,131
Trainable params: 13,489,131
Non-trainable params: 0
_________________________________________________________________
